In [1]:
### 线性回归的简洁实现
# start coding at 01-21 12:15 on Mac

In [2]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

# 生成数据集

In [3]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.randn(num_examples, num_inputs, dtype=torch.float32)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float32)

# 读取数据

In [14]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量数据
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

# 定义模型

In [6]:
import torch.nn as nn

class LinearNet(nn.Module):
        def __init__(self, n_feature):
            super(LinearNet, self).__init__()
            self.linear = nn.Linear(n_feature, 1)
            
            
        # forward 定义前向传播
        def forward(self, x):
            y = self.linear(x)
            return y
        
        
net  = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


# 初始化模型参数

In [7]:
from torch.nn import init


init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

# 定义损失函数

In [8]:
loss = nn.MSELoss()

# 定义优化算法

In [11]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.03)

print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


# 训练模型

In [15]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X,y in data_iter:
        output = net(X)
        # 这个就是和numpy里的reshape是一样的
        # output不改变数据，但是改变数据的形状，1是表示一列，最后得到的是一个列tensor
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()
        l.backward()
        
        # 按照小批量随机梯度下降的定义，我们在step函数中指明批量大小，从而对批量中样本梯度求平均。
        optimizer.step()
    print('epoch %d, loss : %f' % (epoch, l.item()))

epoch 1, loss : 0.000110
epoch 2, loss : 0.000064
epoch 3, loss : 0.000078


In [18]:
###
# 我们从net获得需要的层，并访问其权重（weight）和偏差（bias）。
# 学到的参数和真实的参数很接近。

dense = net.linear
print(true_b, dense.bias)
print(true_w, dense.weight)

4.2 Parameter containing:
tensor([4.1999], requires_grad=True)
[2, -3.4] Parameter containing:
tensor([[ 1.9999, -3.4004]], requires_grad=True)


In [ ]:
# finished at 14:45